In [1]:
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast
import nltk
import json
import requests
import pandas as pd

from os.path import exists
from os.path import isdir

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ucloud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ucloud/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
model = BloomForCausalLM.from_pretrained("bigscience/bloom-7b1") #remove -1b7 for whole modelb
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-7b1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
#prompt = "The train drove directly down in"
#result_length = 20
#inputs = tokenizer(prompt, return_tensors="pt") #??

name = 'bloom_7b1_validation2'

In [4]:
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_length=result_length
                      )[0]))

NameError: name 'inputs' is not defined

# Load train and validation data

In [4]:
def error(msg):
    print('  [\033[91mx\033[0m] ' + msg)
    exit(1)

def success(msg):
    print('  [\033[92mo\033[0m] ' + msg)
    
def load_json_lines(f):
    if not exists(f):
        error('The file "' + f + '" does not exist.')

    ret = []
    num = 1
    
    if isdir(f):
        f = glob(f + '/*.json*')
        
        if len(f) != 1:
            error('The input is an directory that contains multiple json files. Please create only a single json file. Got ' + str(f))
        
        f = f[0]
    
    with open(f, 'r') as inp:
        for l in inp:
            try:
                ret += [json.loads(l)]
            except:
                error('Invalid line ' + str(num) + ' in "' + f + '" with content: ' + l.strip())
            num += 1

    success('The file ' + f + ' is in JSONL format.')
    return ret

In [5]:
train = load_json_lines('train.jsonl')
validation = load_json_lines('validation.jsonl')
test = load_json_lines('input.jsonl')

  [o] The file train.jsonl is in JSONL format.
  [o] The file validation.jsonl is in JSONL format.
  [o] The file input.jsonl is in JSONL format.


In [ ]:
validation[130]['targetDescription'][-3:]

In [ ]:
i = 2
validation[i]['targetTitle'], validation[i]['postText'], validation[i]['targetDescription'], validation[i]['targetKeywords']

In [ ]:
len(' '.join(validation[3]['targetParagraphs']))

In [ ]:
"abcdefghijkl"[-5:]

In [6]:
#Function for creating prompt from entry

def make_prompt(entry):
    article = ' '.join(entry['targetParagraphs'])
    title = entry['targetTitle']
    description = entry['targetDescription']
    tags = entry['targetKeywords']
    clickbait = entry['postText'][0]
    
    words = ['what', 'why']
    
    if len(article) > 4100:
        article = article[:2500] + article[-1000:]
    
    if title.lower() == clickbait.lower():
        title = ''
    else:
        title = f"\n\nArticle title: {title}"
    
    if tags:
        tags = f"\n\nTags: {', '.join(tags.split(','))}"
    else:
        tags = ''
    
    if description and description[-3:] != '...':
        description = f"\n\nDescription: {description}"
    else:
        description = ''
    
    if '?' in clickbait:
        clickbait = f"\n\nQuestion: {clickbait}\n\nAnswer:"
    elif [x for x in words if x in clickbait.lower()]:
        clickbait = f"\n\n{clickbait}:"
    else:
        clickbait = f"\n\nStatement to be answered: {clickbait}\n\nAnswer:"
    
    
    prompt = f"{title}{description}\n\nArticle: {article}{tags}{clickbait}"
    return prompt.strip()

print(make_prompt(validation[3]))

Article title: Alan Rickman & Rupert Grint On 'CBGB,' Reuniting Post-'Potter' And Favorite Halloween Costumes

Article: The mythology of punk music's evolution can be traced back, more or less, to one singular staple: CBGB. Opened in 1973 in downtown Manhattan's East Village, the nightclub that originally intended to house country, bluegrass and blues music (hint its acronym) quickly became a breeding ground for bands like The Ramones, Talking Heads and The Police, among countless others. The legacy of the club, which closed in 2006 over a rent dispute, is on display in a new movie that stars Alan Rickman as bemused owner Hilly Kristal. It's a bit of a departure for anyone more accustomed to seeing Rickman, 67, play sinister Severus Snape in "Harry Potter" or Judge Turpin in "Sweeney Todd." The movie also afforded him the opportunity to reunite with fellow Hogwarts alum Rupert Grint, who portrays one-third of derriere-bearing punk band The Dead Boys, alongside Justin Bartha and Bronson

In [7]:
predictions_from_prompts = []
prompts = [make_prompt(x) for x in validation]

for i, prompt in enumerate(prompts):
    print(i,"/",len(prompts))
    inputs = tokenizer(prompt, return_tensors="pt")
    prediction = tokenizer.decode(model.generate(inputs["input_ids"], 
                           max_length=len(inputs['input_ids'][0])+20
                          )[0])
    anchor = prompt[-15:]
    pred = prediction[prediction.find(anchor)+15:].strip('</s>').strip()
    if '\n' in pred:
        #pred = pred.replace('\n', ' ')
        pred = [x.strip() for x in pred.split('\n') if len(x)!=0]
    predictions_from_prompts.append(pred)
    print('title:',test[i]['postText'][0],'\nprediction:',pred)

0 / 800
title: This simple household item saves lives 
prediction: The game is being delayed because it’s too dark.
1 / 800
title: You'll Never Believe Who Robert Downey Jr.'s 'Free Pass' Is! 
prediction: ['intentionally', 'Article: © REUTERS/Laura Segall Maricopa County Sheriff Joe Arpaio']
2 / 800
title: THE NEWEST THEORY ON HIDDLESWIFT'S ROMANCE IS BIZARRE BUT STRANGELY COMPELLING 
prediction: ['Here’s how much you should be tipping your hairdresser:', '20% is standard and']
3 / 800
title: NBA Star surprises an inner-city youth basketball team 
prediction: The mythology of punk music's evolution can be traced back, more or less, to one singular
4 / 800
title: The little girl from "Titanic" is all grown up and still crazy adorable 
prediction: The man, who was not named, swallowed a 64GB microSD card and then po
5 / 800
title: Why we never really get over that first love 
prediction: The study was published in the journal Alcoholism: Clinical and Experimental Research. The researcher

In [23]:
test[662:][0]

{'uuid': '00b53907-b1f2-4588-9578-e4aafa09446c',
 'postId': '4od57g',
 'postText': ['She Fills Her Coffee Pot With Ramen Noodles... But What Happens Next? GENIUS!!'],
 'postPlatform': 'reddit',
 'targetParagraphs': ['While the combination of ramen noodles and coffee usually don’t go together, the making of those noodles within a coffee pot offers up a unique way to cook them. To begin the process, you should simply crumble up the noodles within a package, since the smaller size will make it easier to heat the product up.',
  'From there, place the noodles in the coffee pot, making sure that all of the noodles are exposed. Next, bring together any other items you might want to add and then fill the pot with a few cups of water. That’s followed by then turning the pot on, with the process lasting anywhere between two to three minutes. Once that’s been completed, the food should be ready, with the added items adding some flavoring to the overall mix. One important thing to remember is tha

In [ ]:
#Fjern gentagelser fra predictions
#Fjern overskrifter fra predictions
#Fjern A: fra predictions
#Fjern Explanation: fra predictions
#Fjern Reasoning: fra predictions

In [ ]:
def allow(pred, prev_pred):
    
    bad_words = ['a:', "'a:'", 'reason:', 'reasoning:', 'explanation:', 'answer:',
             "'answer:'", 'description:', 'i think the', 'says:', 'article:', 'source:', 'http',
                'question:', 'article title:']
    
    pred = pred.lower()
    prev_pred = prev_pred.lower()
    if len(pred) == 0:
        return False
    if any(x in pred for x in bad_words):
        return False
    if pred == prev_pred:
        return False
    return True

In [ ]:
with open('bloom_7b1_test1.txt', 'r', encoding="utf-8") as f:
    predictions = [line.strip('\n').strip('[]').split(',') for line in f]

pred_count = 0
clean_predictions = []
for i, preds in enumerate(predictions):
    tmp_pred = []
    prev_pred = ""
    for pred in preds:
        pred = pred.strip()
        if allow(pred, prev_pred):
            pred_count += 1
            tmp_pred.append(pred)
        prev_pred = pred
    clean_predictions.append(tmp_pred)

pred_count, clean_predictions

In [52]:
with open("BLOOM_7b1_test_predictions.txt", 'r', encoding='utf-8') as f:
    predictions = [line.strip('\n').strip('[]') for line in f]

clean_predictions = []
for line in dat:
    
    line = line.strip('prediction: ')
    line = line.replace('"','')
    line = line.replace("'",'')
        
    if '[' in line:
        line = line.strip("[]")
        line = line.split(', ')
    else:
        line = [line]

    clean_predictions.append(line)

In [53]:
clean_predictions

[['Erin Zaikis, 24, is a Boston native who spent a summer in Mumbai, India'],
 ['Gwyneth Paltrow'],
 ['Statement to be answered: THE NEWEST THEORY ON HIDDLESWIFTS ROM'],
 ['JaVale McGee, Denver Nuggets center, and his mother, Pam McGee, were inspired to spons'],
 ['The little girl from Titanic is all grown up and still crazy adorable, The'],
 ['I think the reason we never really get over that first love is because we never really get ov'],
 ['The reason why you should never line the toilet seat with paper is because it is a breeding ground f'],
 [''],
 ['Ratan Tata is heartbroken because he is lonely and has a painful time amid ongoing tussl'],
 ['11 great movies from 2016 that you can stream on Netflix (and 1 on Hulu), Statement to be'],
 ['The Galaxy S7 is a great phone, but it doesnt have the battery life to last th'],
 ['It is with a heavy heart that I must share the sad news of the passing of a great beas'],
 ['Chicago, I think the answer is, Chicago, Because, The city is the larg

In [8]:
df = pd.read_json('validation.jsonl', lines=True)

In [10]:
spoilers = df[["uuid","postId"]]
spoilers['spoiler'] = predictions_from_prompts

json_output = spoilers.to_json(orient='records', lines=True)

with open(f"{name}.jsonl", 'w') as f:
    f.write(json_output)

/tmp/ipykernel_3149/4068465797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spoilers['spoiler'] = predictions_from_prompts


In [61]:
!python3 evaluation.py --input_run bloom_7b1_test2.jsonl --task 2

  [o] The file bloom_7b1_test2.jsonl is in JSONL format.
  [o] Spoiler generations have correct format. Found 1000
  [o] No ground-truth is passed. I tested the input run and the input run is valid.


In [ ]:
!pip install torch